# Basic Stock Analysis

How to evaluate the performance of your assets?
Optimize your portfolio by reviewing the statistics of your stocks.

### 1. Importing Libraries

In [2]:
import pandas_datareader as pdr
import numpy as np
import datetime as dt
import pandas as pd
from sklearn.linear_model import LinearRegression

### 2. Calculating Beta
Beta expresses how "dynamic" an asset reacts to market fluctuations. A high beta means high risk, but also high opportunity, depending on the market.
- Beta < 1: More stable, but also less growth opportunity
- Beta > 1: More risky, but also more gain opportunity

#### Method
- Get raw stock data of the market and the stocks of interest (can be 1 year, 3 years or other timeframe). Typically the S&P500 (Symbol "^GSPC") is used as the representation of the market data.
- Transform raw data into a dataframe with returns
- Calculate Beta
- Calculate the expected rate of return (CAPM method)
- Interprete the results

In [47]:
def calc_return(df):
    '''Function to generate a detaframe with returns
    - Input: Dataframe with raw stock data
    - Output: Dataframe with returns'''

    x = df.copy()
    
    for i in range(1, len(df)):
        x.iloc[i] = (df.iloc[i] - df.iloc[i-1]) / df.iloc[i-1] * 100

    x.iloc[0] = 0

    return x

# Alternative way:
# log_returns = np.log(data / data.shift())

In [75]:
tickers = ['XIC.TO', '^GSPC', 'SHOP', 'TSLA', 'VFV.TO']
start = dt.datetime(2018, 12, 24)
end = dt.datetime(2021, 12, 24)
df_stocks = pdr.get_data_yahoo(tickers, start, end, interval="d")

df_stocks = df_stocks['Adj Close']

df_stocks

Symbols,XIC.TO,^GSPC,SHOP,TSLA,VFV.TO
Date,,,,,
2018-12-24,20.099087,2351.100098,119.099998,59.077999,54.506882
2018-12-26,NaN,2467.699951,130.210007,65.218002,NaN
2018-12-27,20.646492,2488.830078,132.399994,63.226002,57.672108
2018-12-28,20.764044,2485.739990,133.460007,66.774002,57.985279
2018-12-31,20.929348,2506.850098,138.449997,66.559998,58.090946
...,...,...,...,...,...
2021-12-20,32.810001,4568.020020,1282.170044,899.940002,105.099998
2021-12-21,33.419998,4649.229980,1370.109985,938.530029,106.750000
2021-12-22,33.660000,4696.560059,1385.630005,1008.869995,107.150002


In [76]:
# generating dataframe with returns by using the alternative way
df_returns = np.log(df_stocks / df_stocks.shift())

### Calculating Beta

In [77]:
cov = log_returns.cov()
var = log_returns['^GSPC'].var()
beta_xic = cov.loc['XIC.TO', '^GSPC'] / var
beta_shop = cov.loc['SHOP', '^GSPC'] / var
beta_tsla = cov.loc['TSLA', '^GSPC'] / var
beta_vfv = cov.loc['VFV.TO', '^GSPC'] / var

In [78]:
print('Betas for XIC: {:1.2}, SHOPIFY: {:1.2}, Tesla: {:1.2}, Vanguard: {:1.2}'.format(beta_xic, beta_shop, beta_tsla, beta_vfv))

Betas for XIC: 0.79, SHOPIFY: 1.2, Tesla: 1.3, Vanguard: 0.88
